In [14]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import layers

import tensorflow as tf
import numpy as np
import pandas as pd
from layers import conv_layer, max_pool_2x2, full_layer
import datetime



Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [58]:
def model_fn(x, target, mode, params):
    y_ = tf.cast(target, tf.float32)
    x_image = tf.reshape(x, [-1, 28, 28, 1])

    # 합성곱 계층 1
    conv1 = layers.convolution2d(x_image, 32, [5,5],
                                activation_fn=tf.nn.relu,
                                biases_initializer=tf.constant_initializer(0.1),
                                weights_initializer=tf.truncated_normal_initializer(stddev=0.1))
    pool1 = layers.max_pool2d(conv1, [2,2])

    # 합성곱 계층 2
    conv2 = layers.convolution2d(pool1, 64, [5,5],
                                activation_fn=tf.nn.relu,
                                biases_initializer=tf.constant_initializer(0.1),
                                weights_initializer=tf.truncated_normal_initializer(stddev=0.1))
    pool2 = layers.max_pool2d(conv2, [2,2])
    print(pool2.get_shape())

    # 완전연결계층
    pool2_flat = tf.reshape(pool2, [-1, 7*7*64])
    fc1 = layers.fully_connected(pool2_flat, 1024,
                                activation_fn=tf.nn.relu,
                                biases_initializer=tf.constant_initializer(0.1),
                                weights_initializer=tf.truncated_normal_initializer(stddev=0.1))
    fc1_drop = layers.dropout(fc1, keep_prob=model_params["dropout"],
                             is_training=(mode == 'train'))
    print(fc1_drop)

    # 판독 계층
    y_conv = layers.fully_connected(fc1_drop, 10,
                                activation_fn=None)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
    train_op = tf.contrib.layers.optimize_loss(loss=cross_entropy,
                                              global_step=tf.contrib.framework.get_global_step(),
                                              learning_rate=params["learning_rate"],
                                              optimizer="Adam")
    predictions = tf.argmax(y_conv,1)
    return predictions, cross_entropy, train_op

In [59]:
DATA_DIR = 'tmp/data'
MINIBATCH_SIZE = 50
STEPS = 5000


# mnist[0]은 이미지, mnist[1]은 숫자값 ont Hot
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting tmp/data/train-images-idx3-ubyte.gz
Extracting tmp/data/train-labels-idx1-ubyte.gz
Extracting tmp/data/t10k-images-idx3-ubyte.gz
Extracting tmp/data/t10k-labels-idx1-ubyte.gz


In [63]:
x_data, y_data = mnist.train.images, np.int32(mnist.train.labels)
tf.cast(x_data, tf.float32)
tf.cast(y_data, tf.float32)

model_params = {"learning_rate": 1e-4, "dropout": 0.5}

CNN = tf.contrib.learn.Estimator(model_fn = model_fn,
                                params = model_params)
print("Starting training for %s steps max"% 500)

CNN.fit(x=mnist.train.images, 
        y=mnist.train.labels, 
        batch_size=50,
        max_steps=500)

test_acc = 0

for ii in range(5):
    batch = mnist.test.next_batch(2000)
    predictions = list(CNN.predict(batch[0], as_iterable=True))
    test_acc = test_acc + (np.argmax(batch[1],1) == predictions).mean()
    
print(test_acc/5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7857b93d30>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpn2ul7z7e'}
Starting training for 500 steps max
(?, 7, 7, 64)
Tensor("Dropout/dropout_1/mul:0", shape=(?, 1024), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_ini

In [42]:
x_data, y_data = mnist.train.images, np.int32(mnist.train.labels)

In [43]:
x_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [45]:
y_data

<tf.Tensor 'Cast_8:0' shape=(55000, 10) dtype=float32>